In [1]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
from pyspark.streaming.kafka import KafkaUtils
import pyspark.sql.functions as f
import json
import re
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import sys
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_json
from pyspark.sql.functions import lit
import pickle
import keras
from numpy import zeros

Using TensorFlow backend.


In [2]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier, DecisionTreeClassifier, RandomForestClassifier
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.ml.regression import IsotonicRegression
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [4]:
spark

In [5]:
sql_context = SQLContext(sc)
sql_context

In [7]:
a1 = spark.read.csv("hdfs:///user/spark/datafile/1.csv", header=True, inferSchema=True)
a2 = spark.read.csv("hdfs:///user/spark/datafile/2.csv", header=True, inferSchema=True)
a3 = spark.read.csv("hdfs:///user/spark/datafile/3.csv", header=True, inferSchema=True)
a4 = spark.read.csv("hdfs:///user/spark/datafile/4.csv", header=True, inferSchema=True)
a5 = spark.read.csv("hdfs:///user/spark/datafile/5.csv", header=True, inferSchema=True)
a6 = spark.read.csv("hdfs:///user/spark/datafile/6.csv", header=True, inferSchema=True)
a7 = spark.read.csv("hdfs:///user/spark/datafile/7.csv", header=True, inferSchema=True)
a8 = spark.read.csv("hdfs:///user/spark/datafile/8.csv", header=True, inferSchema=True)
a9 = spark.read.csv("hdfs:///user/spark/datafile/9.csv", header=True, inferSchema=True)
a10 = spark.read.csv("hdfs:///user/spark/datafile/10.csv", header=True, inferSchema=True)

In [8]:
#loaded_data = csi_pos_neg.repartition(10000)
loaded_data.rdd.getNumPartitions()

67

In [9]:
loaded_data.count()

2403430

In [10]:
cnt = loaded_data.select('DEVICE_ID').distinct().count()

In [11]:
cnt

73

In [12]:
loaded_data.dtypes

[('TIMESTAMP', 'timestamp'),
 ('누적전력량', 'double'),
 ('무효전력평균', 'double'),
 ('상전압평균', 'double'),
 ('선간전압평균', 'double'),
 ('역률평균', 'double'),
 ('온도', 'double'),
 ('유효전력평균', 'double'),
 ('전류고조파평균', 'double'),
 ('전류평균', 'double'),
 ('전압고조파평균', 'double'),
 ('주파수', 'double'),
 ('DEVICE_ID', 'int'),
 ('LABEL_역률평균', 'int'),
 ('LABEL_전압고조파평균', 'int'),
 ('LABEL_전류고조파평균', 'int')]

In [11]:
a1 = a1.dropna(how='any')
a2 = a2.dropna(how='any')
a3 = a3.dropna(how='any')
a4 = a4.dropna(how='any')
a5 = a5.dropna(how='any')
a6 = a6.dropna(how='any')
a7 = a7.dropna(how='any')
a8 = a8.dropna(how='any')
a9 = a9.dropna(how='any')
a10 = a10.dropna(how='any')

In [14]:
loaded_data.count()

2403429

In [50]:
loaded_data.groupBy("DEVICE_ID").count().orderBy(col("count").desc()).show(truncate=False)

+---------+-----+
|DEVICE_ID|count|
+---------+-----+
|2257     |90697|
|2259     |90654|
|2258     |89280|
|1289     |53272|
|1288     |49574|
|2297     |43200|
|1385     |42983|
|1287     |34482|
|1290     |32019|
|7255     |28800|
|7247     |28800|
|7253     |28800|
|5764     |28800|
|5754     |28800|
|7277     |26715|
|5640     |24480|
|7301     |24480|
|7303     |24480|
|7299     |24480|
|6521     |20160|
+---------+-----+
only showing top 20 rows



In [13]:
# import pickle
# with open("/root/rounded_df", "rb") as file:
#     df = pickle.load(file)

In [14]:
# df = spark.createDataFrame(loaded_data)

In [15]:
# df.show(1)

In [16]:
loaded_data.show(5)

+-------------------+----------+------------+----------+------------+--------+------+------------+--------------+--------+--------------+---------+---------+--------------+--------------------+--------------------+----------+
|          TIMESTAMP|누적전력량|무효전력평균|상전압평균|선간전압평균|역률평균|  온도|유효전력평균|전류고조파평균|전류평균|전압고조파평균|   주파수|DEVICE_ID|LABEL_역률평균|LABEL_전압고조파평균|LABEL_전류고조파평균|LABEL_NAME|
+-------------------+----------+------------+----------+------------+--------+------+------------+--------------+--------+--------------+---------+---------+--------------+--------------------+--------------------+----------+
|2021-01-19 00:00:48| 1523.2915|         0.0|132.916672|  230.083328|     0.0| 38.75|         0.0|           0.0|     0.0|      3.287761|59.835415|     7303|             2|                   0|                   0|       1.0|
|2021-01-19 00:01:48| 1523.2915|         0.0|131.916672|  228.416672|     0.0|39.375|         0.0|           0.0|     0.0|      3.190104|59.835415|     7303|            

In [78]:
# df.columns

In [12]:
loaded_data = loaded_data.drop(col("LABEL_NAME"))

In [13]:
loaded_data.columns

['TIMESTAMP',
 '누적전력량',
 '무효전력평균',
 '상전압평균',
 '선간전압평균',
 '역률평균',
 '온도',
 '유효전력평균',
 '전류고조파평균',
 '전류평균',
 '전압고조파평균',
 '주파수',
 'DEVICE_ID',
 'LABEL_역률평균',
 'LABEL_전압고조파평균',
 'LABEL_전류고조파평균']

In [13]:
idindexer1 = StringIndexer(inputCol='DEVICE_ID', outputCol="indexedDEVICE_ID").fit(a1)
idindexer2 = StringIndexer(inputCol='DEVICE_ID', outputCol="indexedDEVICE_ID").fit(a2)
idindexer3 = StringIndexer(inputCol='DEVICE_ID', outputCol="indexedDEVICE_ID").fit(a3)
idindexer4 = StringIndexer(inputCol='DEVICE_ID', outputCol="indexedDEVICE_ID").fit(a4)
idindexer5 = StringIndexer(inputCol='DEVICE_ID', outputCol="indexedDEVICE_ID").fit(a5)
idindexer6 = StringIndexer(inputCol='DEVICE_ID', outputCol="indexedDEVICE_ID").fit(a6)
idindexer7 = StringIndexer(inputCol='DEVICE_ID', outputCol="indexedDEVICE_ID").fit(a7)
idindexer8 = StringIndexer(inputCol='DEVICE_ID', outputCol="indexedDEVICE_ID").fit(a8)
idindexer9 = StringIndexer(inputCol='DEVICE_ID', outputCol="indexedDEVICE_ID").fit(a9)
idindexer10 = StringIndexer(inputCol='DEVICE_ID', outputCol="indexedDEVICE_ID").fit(a10)

In [14]:
id_df1 = idindexer1.transform(a1)
id_df2 = idindexer2.transform(a2)
id_df3 = idindexer3.transform(a3)
id_df4 = idindexer4.transform(a4)
id_df5 = idindexer5.transform(a5)
id_df6 = idindexer6.transform(a6)
id_df7 = idindexer7.transform(a7)
id_df8 = idindexer8.transform(a8)
id_df9 = idindexer9.transform(a9)
id_df10 = idindexer10.transform(a10)

In [22]:
id_df.select("indexedDEVICE_ID").show(50)

+----------------+
|indexedDEVICE_ID|
+----------------+
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|            15.0|
|           

In [15]:
features = ['누적전력량', 'indexedDEVICE_ID', '전압고조파평균', '상전압평균', '선간전압평균', '온도', '무효전력평균', '전류고조파평균', '전류평균', '유효전력평균', '역률평균', '주파수']

In [16]:
va = VectorAssembler(inputCols=features, outputCol='features')

In [17]:
va_df1 = va.transform(id_df1)
va_df2 = va.transform(id_df2)
va_df3 = va.transform(id_df3)
va_df4 = va.transform(id_df4)
va_df5 = va.transform(id_df5)
va_df6 = va.transform(id_df6)
va_df7 = va.transform(id_df7)
va_df8 = va.transform(id_df8)
va_df9 = va.transform(id_df9)
va_df10 = va.transform(id_df10)

In [20]:
va_df

DataFrame[TIMESTAMP: timestamp, 누적전력량: double, 무효전력평균: double, 상전압평균: double, 선간전압평균: double, 역률평균: double, 온도: double, 유효전력평균: double, 전류고조파평균: double, 전류평균: double, 전압고조파평균: double, 주파수: double, DEVICE_ID: int, LABEL_역률평균: int, LABEL_전압고조파평균: int, LABEL_전류고조파평균: int, indexedDEVICE_ID: double, features: vector]

In [18]:
label_total = ['LABEL_역률평균', 'LABEL_전압고조파평균', 'LABEL_전류고조파평균']

In [21]:
va2 = VectorAssembler(inputCols=label_total, outputCol='LABEL_NAME2')

In [22]:
va2_df1 = va2.transform(va_df1)
va2_df2 = va2.transform(va_df2)
va2_df3 = va2.transform(va_df3)
va2_df4 = va2.transform(va_df4)
va2_df5 = va2.transform(va_df5)
va2_df6 = va2.transform(va_df6)
va2_df7 = va2.transform(va_df7)
va2_df8 = va2.transform(va_df8)
va2_df9 = va2.transform(va_df9)
va2_df10 = va2.transform(va_df10)

In [21]:
va2_df.select("LABEL_NAME").show(5)

+-------------+
|   LABEL_NAME|
+-------------+
|[2.0,0.0,0.0]|
|[2.0,0.0,0.0]|
|[2.0,0.0,0.0]|
|[2.0,0.0,0.0]|
|[2.0,0.0,0.0]|
+-------------+
only showing top 5 rows



In [31]:
va2_df.groupBy("LABEL_NAME").count().orderBy(col("count").asc()).show()

+-------------+-----+
|   LABEL_NAME|count|
+-------------+-----+
|[1.0,1.0,2.0]|  100|
|[1.0,2.0,2.0]|  156|
|[1.0,0.0,2.0]|  308|
|[0.0,2.0,2.0]|  498|
|[2.0,2.0,2.0]|  979|
|[0.0,1.0,2.0]|  989|
|[2.0,1.0,2.0]| 1498|
|[1.0,1.0,1.0]| 2023|
|[0.0,0.0,2.0]| 2694|
|[2.0,0.0,2.0]| 3954|
|[1.0,0.0,1.0]| 4601|
|[1.0,2.0,1.0]| 6380|
|[1.0,0.0,0.0]|10965|
|[0.0,1.0,1.0]|11795|
|[2.0,1.0,1.0]|12071|
|[0.0,2.0,1.0]|13760|
|[2.0,2.0,1.0]|14269|
|[2.0,0.0,1.0]|14828|
|[0.0,0.0,1.0]|17608|
|[1.0,1.0,0.0]|19622|
+-------------+-----+
only showing top 20 rows



In [23]:
va2_df.printSchema()

root
 |-- TIMESTAMP: timestamp (nullable = true)
 |-- 누적전력량: double (nullable = true)
 |-- 무효전력평균: double (nullable = true)
 |-- 상전압평균: double (nullable = true)
 |-- 선간전압평균: double (nullable = true)
 |-- 역률평균: double (nullable = true)
 |-- 온도: double (nullable = true)
 |-- 유효전력평균: double (nullable = true)
 |-- 전류고조파평균: double (nullable = true)
 |-- 전류평균: double (nullable = true)
 |-- 전압고조파평균: double (nullable = true)
 |-- 주파수: double (nullable = true)
 |-- DEVICE_ID: integer (nullable = true)
 |-- LABEL_역률평균: integer (nullable = true)
 |-- LABEL_전압고조파평균: integer (nullable = true)
 |-- LABEL_전류고조파평균: integer (nullable = true)
 |-- indexedDEVICE_ID: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- LABEL_NAME: vector (nullable = true)



In [24]:
labelIndexer1 = VectorIndexer(inputCol='LABEL_NAME2', outputCol="indexedLabel", maxCategories=28).fit(va2_df1)
labelIndexer2 = VectorIndexer(inputCol='LABEL_NAME2', outputCol="indexedLabel", maxCategories=28).fit(va2_df2)
labelIndexer3 = VectorIndexer(inputCol='LABEL_NAME2', outputCol="indexedLabel", maxCategories=28).fit(va2_df3)
labelIndexer4 = VectorIndexer(inputCol='LABEL_NAME2', outputCol="indexedLabel", maxCategories=28).fit(va2_df4)
labelIndexer5 = VectorIndexer(inputCol='LABEL_NAME2', outputCol="indexedLabel", maxCategories=28).fit(va2_df5)
labelIndexer6 = VectorIndexer(inputCol='LABEL_NAME2', outputCol="indexedLabel", maxCategories=28).fit(va2_df6)
labelIndexer7 = VectorIndexer(inputCol='LABEL_NAME2', outputCol="indexedLabel", maxCategories=28).fit(va2_df7)
labelIndexer8 = VectorIndexer(inputCol='LABEL_NAME2', outputCol="indexedLabel", maxCategories=28).fit(va2_df8)
labelIndexer9 = VectorIndexer(inputCol='LABEL_NAME2', outputCol="indexedLabel", maxCategories=28).fit(va2_df9)
labelIndexer10 = VectorIndexer(inputCol='LABEL_NAME2', outputCol="indexedLabel", maxCategories=28).fit(va2_df10)

In [26]:
df_med1 = labelIndexer1.transform(va2_df1)
df_med2 = labelIndexer2.transform(va2_df2)
df_med3 = labelIndexer3.transform(va2_df3)
df_med4 = labelIndexer4.transform(va2_df4)
df_med5 = labelIndexer5.transform(va2_df5)
df_med6 = labelIndexer6.transform(va2_df6)
df_med7 = labelIndexer7.transform(va2_df7)
df_med8 = labelIndexer8.transform(va2_df8)
df_med9 = labelIndexer9.transform(va2_df9)
df_med10 = labelIndexer10.transform(va2_df10)

In [24]:
df_med.select("indexedLabel").show(10)

+-------------+
| indexedLabel|
+-------------+
|[2.0,0.0,0.0]|
|[2.0,0.0,0.0]|
|[2.0,0.0,0.0]|
|[2.0,0.0,0.0]|
|[2.0,0.0,0.0]|
|[2.0,0.0,0.0]|
|[2.0,0.0,0.0]|
|[2.0,0.0,0.0]|
|[2.0,1.0,0.0]|
|[2.0,1.0,0.0]|
+-------------+
only showing top 10 rows



In [28]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import col

dense_format_udf = udf(lambda x: str(x), StringType())

df1 = df_med1.withColumn('indexedLabel', dense_format_udf(col('LABEL_NAME2')))
df2 = df_med2.withColumn('indexedLabel', dense_format_udf(col('LABEL_NAME2')))
df3 = df_med3.withColumn('indexedLabel', dense_format_udf(col('LABEL_NAME2')))
df4 = df_med4.withColumn('indexedLabel', dense_format_udf(col('LABEL_NAME2')))
df5 = df_med5.withColumn('indexedLabel', dense_format_udf(col('LABEL_NAME2')))
df6 = df_med6.withColumn('indexedLabel', dense_format_udf(col('LABEL_NAME2')))
df7 = df_med7.withColumn('indexedLabel', dense_format_udf(col('LABEL_NAME2')))
df8 = df_med8.withColumn('indexedLabel', dense_format_udf(col('LABEL_NAME2')))
df9 = df_med9.withColumn('indexedLabel', dense_format_udf(col('LABEL_NAME2')))
df10 = df_med10.withColumn('indexedLabel', dense_format_udf(col('LABEL_NAME2')))

In [26]:
df.printSchema()

root
 |-- TIMESTAMP: timestamp (nullable = true)
 |-- 누적전력량: double (nullable = true)
 |-- 무효전력평균: double (nullable = true)
 |-- 상전압평균: double (nullable = true)
 |-- 선간전압평균: double (nullable = true)
 |-- 역률평균: double (nullable = true)
 |-- 온도: double (nullable = true)
 |-- 유효전력평균: double (nullable = true)
 |-- 전류고조파평균: double (nullable = true)
 |-- 전류평균: double (nullable = true)
 |-- 전압고조파평균: double (nullable = true)
 |-- 주파수: double (nullable = true)
 |-- DEVICE_ID: integer (nullable = true)
 |-- LABEL_역률평균: integer (nullable = true)
 |-- LABEL_전압고조파평균: integer (nullable = true)
 |-- LABEL_전류고조파평균: integer (nullable = true)
 |-- indexedDEVICE_ID: double (nullable = false)
 |-- LABEL_NAME: vector (nullable = true)
 |-- indexedLabel: string (nullable = true)



In [27]:
# from pyspark.mllib.util import MLUtils
# from pyspark.ml.feature import StandardScaler
# scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [28]:
# df = scaler.fit(df).transform(df)

In [29]:
label_aa = StringIndexer(inputCol='indexedLabel', outputCol="label").fit(df)

In [35]:
temp1 = label_aa.fit(df1).transform(df1)
temp2 = label_aa.fit(df2).transform(df2)
temp3 = label_aa.fit(df3).transform(df3)
temp4 = label_aa.fit(df4).transform(df4)
temp5 = label_aa.fit(df5).transform(df5)
temp6 = label_aa.fit(df6).transform(df6)
temp7 = label_aa.fit(df7).transform(df7)
temp8 = label_aa.fit(df8).transform(df8)
temp9 = label_aa.fit(df9).transform(df9)
temp10 = label_aa.fit(df10).transform(df10)

In [30]:
temp = label_aa.transform(df)

In [32]:
temp.groupBy("label").count().orderBy(col("count").desc()).show(truncate=False)

+-----+------+
|label|count |
+-----+------+
|0.0  |181956|
|1.0  |173408|
|2.0  |119804|
|3.0  |81100 |
|4.0  |54057 |
|5.0  |36171 |
|6.0  |33655 |
|7.0  |19622 |
|8.0  |17608 |
|9.0  |14828 |
|10.0 |14269 |
|11.0 |13760 |
|12.0 |12071 |
|13.0 |11795 |
|14.0 |10965 |
|15.0 |6380  |
|16.0 |4601  |
|17.0 |3954  |
|18.0 |2694  |
|19.0 |2023  |
+-----+------+
only showing top 20 rows



In [36]:
temp.dtypes

[('TIMESTAMP', 'timestamp'),
 ('누적전력량', 'double'),
 ('무효전력평균', 'double'),
 ('상전압평균', 'double'),
 ('선간전압평균', 'double'),
 ('역률평균', 'double'),
 ('온도', 'double'),
 ('유효전력평균', 'double'),
 ('전류고조파평균', 'double'),
 ('전류평균', 'double'),
 ('전압고조파평균', 'double'),
 ('주파수', 'double'),
 ('DEVICE_ID', 'int'),
 ('LABEL_역률평균', 'int'),
 ('LABEL_전압고조파평균', 'int'),
 ('LABEL_전류고조파평균', 'int'),
 ('indexedDEVICE_ID', 'double'),
 ('LABEL_NAME', 'vector'),
 ('indexedLabel', 'string'),
 ('label', 'double')]

In [40]:
c1 = temp1.drop(col("LABEL_NAME")).drop(col("indexedLabel")).drop(col("indexedDEVICE_ID")).drop(col("LABEL_NAME2")).drop(col("features"))
c2 = temp2.drop(col("LABEL_NAME")).drop(col("indexedLabel")).drop(col("indexedDEVICE_ID")).drop(col("LABEL_NAME2")).drop(col("features"))
c3 = temp3.drop(col("LABEL_NAME")).drop(col("indexedLabel")).drop(col("indexedDEVICE_ID")).drop(col("LABEL_NAME2")).drop(col("features"))
c4 = temp4.drop(col("LABEL_NAME")).drop(col("indexedLabel")).drop(col("indexedDEVICE_ID")).drop(col("LABEL_NAME2")).drop(col("features"))
c5 = temp5.drop(col("LABEL_NAME")).drop(col("indexedLabel")).drop(col("indexedDEVICE_ID")).drop(col("LABEL_NAME2")).drop(col("features"))
c6 = temp6.drop(col("LABEL_NAME")).drop(col("indexedLabel")).drop(col("indexedDEVICE_ID")).drop(col("LABEL_NAME2")).drop(col("features"))
c7 = temp7.drop(col("LABEL_NAME")).drop(col("indexedLabel")).drop(col("indexedDEVICE_ID")).drop(col("LABEL_NAME2")).drop(col("features"))
c8 = temp8.drop(col("LABEL_NAME")).drop(col("indexedLabel")).drop(col("indexedDEVICE_ID")).drop(col("LABEL_NAME2")).drop(col("features"))
c9 = temp9.drop(col("LABEL_NAME")).drop(col("indexedLabel")).drop(col("indexedDEVICE_ID")).drop(col("LABEL_NAME2")).drop(col("features"))
c10 = temp10.drop(col("LABEL_NAME")).drop(col("indexedLabel")).drop(col("indexedDEVICE_ID")).drop(col("LABEL_NAME2")).drop(col("features"))

In [41]:
c1.dtypes

[('TIMESTAMP', 'timestamp'),
 ('누적전력량', 'double'),
 ('무효전력평균', 'double'),
 ('상전압평균', 'double'),
 ('선간전압평균', 'double'),
 ('역률평균', 'double'),
 ('온도', 'double'),
 ('유효전력평균', 'double'),
 ('전류고조파평균', 'double'),
 ('전류평균', 'double'),
 ('전압고조파평균', 'double'),
 ('주파수', 'double'),
 ('DEVICE_ID', 'int'),
 ('LABEL_역률평균', 'int'),
 ('LABEL_전압고조파평균', 'int'),
 ('LABEL_전류고조파평균', 'int'),
 ('label', 'double')]

In [45]:
c1.write.csv("hdfs:///user/spark/datafile/c1.csv")

In [42]:
c1.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("hdfs:///user/spark/datafile/c1.csv")
c2.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("hdfs:///user/spark/datafile/c2.csv")
c3.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("hdfs:///user/spark/datafile/c3.csv")
c4.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("hdfs:///user/spark/datafile/c4.csv")
c5.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("hdfs:///user/spark/datafile/c5.csv")
c6.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("hdfs:///user/spark/datafile/c6.csv")
c7.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("hdfs:///user/spark/datafile/c7.csv")
c8.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("hdfs:///user/spark/datafile/c8.csv")
c9.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("hdfs:///user/spark/datafile/c9.csv")
c10.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("hdfs:///user/spark/datafile/c10.csv")

In [30]:
df.groupBy("indexedLabel").count().orderBy(col("count").desc()).show(truncate=False)

AnalysisException: "cannot resolve '`label`' given input columns: [유효전력평균, 전류평균, LABEL_NAME, LABEL_전류고조파평균, indexedLabel, 역률평균, 전류고조파평균, indexedDEVICE_ID, TIMESTAMP, 전압고조파평균, 무효전력평균, LABEL_전압고조파평균, 선간전압평균, 주파수, 온도, LABEL_역률평균, 누적전력량, 상전압평균, DEVICE_ID];;\n'Aggregate ['label], ['label, count(1) AS count#385L]\n+- Project [TIMESTAMP#10, 누적전력량#11, 무효전력평균#12, 상전압평균#13, 선간전압평균#14, 역률평균#15, 온도#16, 유효전력평균#17, 전류고조파평균#18, 전류평균#19, 전압고조파평균#20, 주파수#21, DEVICE_ID#22, LABEL_역률평균#23, LABEL_전압고조파평균#24, LABEL_전류고조파평균#25, indexedDEVICE_ID#160, LABEL_NAME#181, <lambda>(LABEL_NAME#181) AS indexedLabel#304]\n   +- Project [TIMESTAMP#10, 누적전력량#11, 무효전력평균#12, 상전압평균#13, 선간전압평균#14, 역률평균#15, 온도#16, 유효전력평균#17, 전류고조파평균#18, 전류평균#19, 전압고조파평균#20, 주파수#21, DEVICE_ID#22, LABEL_역률평균#23, LABEL_전압고조파평균#24, LABEL_전류고조파평균#25, indexedDEVICE_ID#160, LABEL_NAME#181, UDF(LABEL_NAME#181) AS indexedLabel#261]\n      +- Project [TIMESTAMP#10, 누적전력량#11, 무효전력평균#12, 상전압평균#13, 선간전압평균#14, 역률평균#15, 온도#16, 유효전력평균#17, 전류고조파평균#18, 전류평균#19

In [98]:
# a = labelIndexer.transform(va_df)
# a.groupBy("indexedLabel").count().orderBy(col("count").desc()).show()

In [28]:
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=cnt+1).fit(df)

In [29]:
from pyspark.ml.feature import PCA
pca_model = PCA(k = 9,inputCol = "indexedFeatures", outputCol = "pca_features")

In [30]:
df.rdd.getNumPartitions()

67

In [31]:
df.count()

2403429

In [32]:
(train, test) = df.randomSplit([0.8, 0.2])

In [33]:
test.rdd.getNumPartitions()

67

In [34]:
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [35]:
from pyspark.ml.classification import DecisionTreeClassifier
gbt = DecisionTreeClassifier(labelCol="label", featuresCol="indexedFeatures", maxBins = cnt+1, maxDepth=30)
# paramGrid = ParamGridBuilder().addGrid(gbt.maxBins,77).addGrid(gbt.maxIter, [4, 6, 8]).addGrid(gbt.lossType, ["entropy", "gini"]).build()

In [106]:
# pipeline = Pipeline(stages=[label_aa, featureIndexer, pca_model, gbt])

In [36]:
pipeline = Pipeline(stages=[label_aa, featureIndexer, gbt])

In [37]:
import time

start = time.time()
model = pipeline.fit(train)
end = time.time()
print(end - start)

KeyboardInterrupt: 

In [40]:
predictions = model.transform(test)

In [79]:
#(train, test) = predictions.randomSplit([0.8, 0.2])

In [ ]:
# CrossValidation

In [66]:
from pyspark.ml.classification import LogisticRegression
#gbt = RandomForestClassifier(labelCol="indexedLabel", featuresCol="pca_features", numTrees = 20)

lr = LogisticRegression(labelCol="indexedLabel", featuresCol="pca_features", maxIter=20, regParam=0.3, elasticNetParam=0)
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.1, 0.3, 0.5]) # regularization parameter
             .addGrid(lr.elasticNetParam, [0.0, 0.1, 0.2]) # Elastic Net Parameter (Ridge = 0)
             .addGrid(lr.maxIter, [10, 20, 50]) #Number of iterations
             .build())

# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=evaluator, \
                    numFolds=5)
cvModel = cv.fit(train)

predictions = cvModel.transform(test)
# Evaluate best model
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(1.0 - accuracy)

IllegalArgumentException: 'Field "pca_features" does not exist.\nAvailable fields: TIMESTAMP, 누적전력량, 무효전력평균, 상전압평균, 선간전압평균, 역률평균, 온도, 유효전력평균, 전류고조파평균, 전류평균, 전압고조파평균, 주파수, DEVICE_ID, LABEL_역률평균, LABEL_전압고조파평균, LABEL_전류고조파평균, indexedDEVICE_ID, features, LABEL_NAME, indexedLabel, CrossValidator_990ca32bcf57_rand'

In [56]:
# Select example rows to display.
predictions.select("prediction", "label", "features").show(100)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[19235.63,6.0,7.2...|
|       0.0|  0.0|[19235.63,6.0,6.9...|
|       0.0|  0.0|[19235.63,6.0,6.3...|
|       0.0|  0.0|[19235.63,6.0,6.5...|
|       0.0|  0.0|[19235.63,6.0,5.8...|
|       0.0|  0.0|[19235.63,6.0,5.3...|
|       0.0|  0.0|[19235.63,6.0,5.4...|
|       0.0|  0.0|[19235.63,6.0,5.8...|
|       0.0|  0.0|[19235.63,6.0,5.9...|
|       0.0|  0.0|[19235.63,6.0,5.8...|
|       0.0|  0.0|[19235.63,6.0,5.7...|
|       0.0|  0.0|[19235.63,6.0,5.6...|
|       0.0|  0.0|[19235.63,6.0,5.4...|
|       0.0|  0.0|[19235.63,6.0,6.3...|
|       0.0|  0.0|[19235.63,6.0,6.7...|
|       0.0|  0.0|[19235.63,6.0,6.6...|
|       0.0|  0.0|[19235.63,6.0,7.1...|
|       0.0|  0.0|[19235.63,6.0,6.7...|
|       0.0|  0.0|[19235.63,6.0,6.7...|
|       0.0|  0.0|[19235.63,6.0,6.6...|
|       0.0|  0.0|[19235.63,6.0,6.4...|
|       0.0|  0.0|[19235.63,6.0,5.5...|


In [57]:
predictions.groupBy("label").count().orderBy(col("count").desc()).show(27)

+-----+-----+
|label|count|
+-----+-----+
|  0.0|36699|
|  1.0|34484|
|  2.0|24258|
|  3.0|16160|
|  4.0|10807|
|  5.0| 7321|
|  6.0| 6777|
|  7.0| 3961|
|  8.0| 3513|
|  9.0| 2902|
| 10.0| 2847|
| 11.0| 2692|
| 13.0| 2413|
| 12.0| 2300|
| 14.0| 2147|
| 15.0| 1287|
| 16.0|  929|
| 17.0|  786|
| 18.0|  514|
| 19.0|  435|
| 20.0|  306|
| 21.0|  229|
| 22.0|  201|
| 23.0|   99|
| 24.0|   71|
| 25.0|   21|
| 26.0|   18|
+-----+-----+



In [58]:
predictions.groupBy("prediction").count().orderBy(col("count").desc()).show(27)

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|36904|
|       1.0|34211|
|       2.0|24331|
|       3.0|16254|
|       4.0|10738|
|       5.0| 7235|
|       6.0| 6789|
|       7.0| 3973|
|       8.0| 3551|
|       9.0| 2888|
|      10.0| 2754|
|      11.0| 2683|
|      12.0| 2386|
|      13.0| 2379|
|      14.0| 2176|
|      15.0| 1293|
|      16.0|  903|
|      17.0|  802|
|      18.0|  543|
|      19.0|  436|
|      20.0|  307|
|      21.0|  219|
|      22.0|  194|
|      23.0|   99|
|      24.0|   70|
|      26.0|   32|
|      25.0|   27|
+----------+-----+



In [41]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")

In [42]:
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.236113


In [43]:
print(accuracy)

0.7638868866514412


In [44]:
# Select (prediction, true label) and compute test error
aa = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="f1")

In [45]:
aa.evaluate(predictions)

0.7634323776083484

In [46]:
# Select (prediction, true label) and compute test error
cc = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="weightedPrecision")

In [47]:
cc.evaluate(predictions)

0.7631222872574142

In [ ]:
gbtModel = model.stages[2]
print(gbtModel)  # summary only